In [117]:
import os
import sys
import requests
import pandas as pd
import plotly.express as px
import matplotlib.pyplot as plt
from datetime import datetime
from dataclasses import dataclass, field
from io import StringIO
import xml.etree.ElementTree as ET

parent_dir = os.path.abspath(os.path.join(os.getcwd(), os.pardir))
sys.path.append(parent_dir)

from functions.display import display

In [118]:
# Treasury API

year = '2023-01-01'
params = {
    'page[size]': '5000',
    'page[number]': '1',
    'filter': f'record_date:gte:{year}',
    'format': 'csv'
}
endpoint = '/v1/accounting/dts/deposits_withdrawals_operating_cash'
base_url = 'https://api.fiscaldata.treasury.gov/services/api/fiscal_service'
url = base_url + endpoint
all_data = []

In [119]:
@dataclass
class Data:
    url: str
    rows: int = 0
    df: pd.DataFrame = None
    last_accessed: datetime = None
    all_data: list = field(default_factory=list)
    params: dict = field(default_factory=dict)
    columns: dict = field(default_factory=dict)
    
    def __post_init__(self):
        """
        Initialize the sub_menu dictionary.
        To add more menu options, create the function and add it to the sub_menu dictionary.
        
        Follow the format shown below. 
        
        Use nested dictionaries to create sub menus.
        """
        self.sub_menu = {
            "Back": 'back',
            "Search": self.search,
            "Parameters": {
                "Back": 'back',
                "List parameters": self.list_parameters,
                "Edit parameters": self.edit_parameters
            },
            "Data": {
                "Back": 'back',
                "View all": self.view_data,
                "View size": self.view_data_size,
                "List columns": self.list_columns
            }
        }
    
    def json_to_df(self, response: requests.Response) -> None:
        data = response.json().get('data', [])
        df = pd.DataFrame(data)
        self.df = pd.concat([self.df, df], ignore_index=True)
        print(f'(JSON) Current progress: {len(self.df)}')
    
    def csv_to_df(self, response: requests.Response) -> None:
        data = pd.read_csv(StringIO(response.content.decode('utf-8')))
        self.df = pd.concat([self.df, data], ignore_index=True)
        print(f'(CSV) Current progress: {len(self.df)}')
    
    #TODO: Add support for XML
    def xml_to_df(self, response: requests.Response) -> None:
        pass
        # root = ET.fromstring(response.content)
        # data = []
        
        # for child in root:
        #     entry = {subchild.tag: subchild.text for subchild in child}
        #     data.append(entry)
        
        # df = pd.DataFrame(data)
        # self.df = pd.concat([self.df, df], ignore_index=True)
        # print(f'(XML) Current progress: {len(self.df)}')
    
    def search(self) -> pd.DataFrame:
        """
        Fetches all data from the API by handling pagination (The amount of data returned at one time).
        """
        while True:
            # Make the request to the API
            response = requests.get(self.url, params=self.params)
            content_type = response.headers.get('content-type')

            # Check if the request was successful
            if response.status_code != 200:
                break
            
            # Extract the data from the response if in JSON
            if 'application/json' in content_type:
                self.json_to_df(response)
            
            # Extract the data from the response if in CSV
            elif 'application/csv' in content_type or 'text/csv' in content_type:
                self.csv_to_df(response)
                
            # Extract the data from the response if in XML
            elif 'application/xml' in content_type or 'text/xml' in content_type:
                print('XML not yet supported')
                break
                #self.xml_to_df(response)
            
            else:
                print('Unknown content type: ' + content_type)
                break

            # Check if there is more data to fetch
            if len(self.df) < int(self.params['page[size]']):
                break

            # Update the parameters for the next request
            self.params['page[number]'] = str(int(self.params['page[number]']) + 1)
            

        return
    
    def list_parameters(self):
        for item in self.params:
            print(item)
    
    #TODO: Add support for editing parameters
    def edit_parameters(self, params):
        pass
        # choice = display(params)
        # self.params = params
    
    def list_columns(self):
        for column in self.columns:
            print(column)
    
    def pick_columns(self):
        return display(self.columns)
    
    def view_data_size(self):
        print(f'Total entries in dataset: {self.rows}')
    
    def view_data(self):
        print(self.df.describe())

In [120]:
data = Data(url=url, params=params)
display(data.sub_menu)


Choose an option:
1. Back
2. Search
3. Parameters
4. Data
(CSV) Current progress: 5000
(CSV) Current progress: 10000
(CSV) Current progress: 15000
(CSV) Current progress: 20000
(CSV) Current progress: 25000
(CSV) Current progress: 30000
(CSV) Current progress: 35000
(CSV) Current progress: 40000
(CSV) Current progress: 45000
(CSV) Current progress: 50000
(CSV) Current progress: 55000
(CSV) Current progress: 60000
(CSV) Current progress: 65000
(CSV) Current progress: 70000
(CSV) Current progress: 75000
(CSV) Current progress: 80000
(CSV) Current progress: 83179

Choose an option:
1. Back
2. Search
3. Parameters
4. Data


In [121]:
data.df

,record_date,account_type,transaction_type,transaction_catg,transaction_catg_desc,transaction_today_amt,transaction_mtd_amt,transaction_fytd_amt,table_nbr,table_nm,src_line_nbr,record_fiscal_year,record_fiscal_quarter,record_calendar_year,record_calendar_quarter,record_calendar_month,record_calendar_day
0,2023-01-03,Treasury General Account (TGA),Deposits,Cash FTD's Received (Table IV),NaN,56761,56761,1026981,II,Deposits and Withdrawals of Operating Cash,1,2023,2,2023,1,1,3
1,2023-01-03,Treasury General Account (TGA),Deposits,Dept of Agriculture (USDA) - misc,NaN,17,17,901,II,Deposits and Withdrawals of Operating Cash,2,2023,2,2023,1,1,3
2,2023-01-03,Treasury General Account (TGA),Deposits,USDA - Commodity Credit Corporation,NaN,31,31,1402,II,Deposits and Withdrawals of Operating Cash,3,2023,2,2023,1,1,3
3,2023-01-03,Treasury General Account (TGA),Deposits,USDA - Federal Crop Insurance Corp Fund,NaN,0,0,2794,II,Deposits and Withdrawals of Operating Cash,4,2023,2,2023,1,1,3
4,2023-01-03,Treasury General Account (TGA),Deposits,USDA - Loan Repayments,NaN,154,154,1807,II,Deposits and Withdrawals of Operating Cash,5,2023,2,2023,1,1,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
83174,2024-11-01,Treasury General Account (TGA),Withdrawals,United States Postal Service (USPS),NaN,125,125,3774,II,Deposits and Withdrawals of Operating Cash,180,2025,1,2024,4,11,1
83175,2024-11-01,Treasury General Account (TGA),Withdrawals,US Army Corps of Engineers,NaN,0,0,2170,II,Deposits and Withdrawals of Operating Cash,181,2025,1,2024,4,11,1
83176,2024-11-01,Treasury General Account (TGA),Withdrawals,Unclassified,NaN,1013,1013,19920,II,Deposits and Withdrawals of Operating Cash,182,2025,1,2024,4,11,1
83177,2024-11-01,Treasury General Account (TGA),Withdrawals,Public Debt Cash Redemp. (Table IIIB),NaN,6610,6610,2747627,II,Deposits and Withdrawals of Operating Cash,184,2025,1,2024,4,11,1
